In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [53]:
df=pd.read_csv('training_set_features.csv')
df2=pd.read_csv('training_set_labels.csv')

In [54]:
df2.drop(['respondent_id'], axis=1, inplace=True)
df=pd.concat([df,df2], axis=1)

In [55]:
df.drop(['employment_industry', 'employment_occupation'], axis=1, inplace=True)

In [48]:
select=df.iloc[:,22:32]
print(select.nunique())

age_group             5
education             4
race                  4
sex                   2
income_poverty        3
marital_status        2
rent_or_own           2
employment_status     3
hhs_geo_region       10
census_msa            3
dtype: int64


In [56]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
for col in df.columns[22:32]:
  df[col]=label_encoder.fit_transform(df[col])

In [57]:
df=df[['doctor_recc_xyz', 'doctor_recc_seasonal', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk', 'health_insurance',
       'opinion_seas_vacc_effective', 'opinion_seas_risk', 'age_group', 'xyz_vaccine', 'seasonal_vaccine']]

In [9]:
df

,doctor_recc_xyz,doctor_recc_seasonal,opinion_xyz_vacc_effective,opinion_xyz_risk,health_insurance,opinion_seas_vacc_effective,opinion_seas_risk,age_group,xyz_vaccine,seasonal_vaccine
0,0.0,0.0,3.0,1.0,1.0,2.0,1.0,3,0,0
1,0.0,0.0,5.0,4.0,1.0,4.0,2.0,1,0,1
2,NaN,NaN,3.0,1.0,NaN,4.0,1.0,0,0,0
3,0.0,1.0,3.0,3.0,NaN,5.0,4.0,4,0,1
4,0.0,0.0,3.0,3.0,NaN,3.0,1.0,2,0,0
...,...,...,...,...,...,...,...,...,...,...
26702,0.0,0.0,3.0,1.0,NaN,5.0,2.0,4,0,0
26703,1.0,1.0,4.0,2.0,1.0,5.0,1.0,0,0,0
26704,0.0,0.0,4.0,4.0,NaN,5.0,4.0,3,0,1
26705,0.0,0.0,3.0,1.0,0.0,2.0,1.0,0,0,0


In [10]:
print(df.isnull().any(axis=1).sum())

12715


In [11]:
print(df.isnull().sum())

doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
health_insurance               12274
opinion_seas_vacc_effective      462
opinion_seas_risk                514
age_group                          0
xyz_vaccine                        0
seasonal_vaccine                   0
dtype: int64


In [62]:
# df.drop(['health_insurance'],axis=1,inplace=True)
df.iloc[:,0:7]=df.iloc[:,0:7].fillna(0)

In [63]:
print(df.isnull().sum())

doctor_recc_xyz                0
doctor_recc_seasonal           0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
age_group                      0
xyz_vaccine                    0
seasonal_vaccine               0
dtype: int64


In [64]:
X1=df.drop(['xyz_vaccine','seasonal_vaccine','age_group'],axis=1)
X2=df.drop(['xyz_vaccine','seasonal_vaccine'],axis=1)
y=df['xyz_vaccine']
y2=df['seasonal_vaccine']

In [65]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X1,y)

GaussianNB()

In [66]:
model.score(X1,y)

0.7963080840229153

In [28]:
df3=pd.read_csv('test_set_features.csv')
df3=df3[['respondent_id', 'doctor_recc_xyz', 'doctor_recc_seasonal', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk', 'health_insurance',
       'opinion_seas_vacc_effective', 'opinion_seas_risk', 'age_group']]

In [29]:
print(df3.isnull().sum())

respondent_id                      0
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
opinion_xyz_vacc_effective       398
opinion_xyz_risk                 380
health_insurance               12228
opinion_seas_vacc_effective      452
opinion_seas_risk                499
age_group                          0
dtype: int64


In [34]:
df3.drop(['health_insurance'], axis=1, inplace=True)

In [32]:
df3['age_group']=label_encoder.fit_transform(df3['age_group'])

In [37]:
df3.iloc[:,1:7]=df3.iloc[:,1:7].fillna(0)

In [39]:
X3=df3.drop(['age_group', 'respondent_id'],axis=1)
X4=df3.drop(['respondent_id'],axis=1)

In [42]:
y_prob=model.predict_proba(X3)

In [43]:
print(y_prob[:,1])

[0.03557047 0.0150952  0.55029851 ... 0.11943528 0.00532722 0.82355751]


In [67]:
model.fit(X2,y)
model.score(X2,y)

0.7965327442243606

In [70]:
y_prob1=model.predict_proba(X4)
print(y_prob1[:,1])

[0.03170854 0.01413086 0.4662029  ... 0.0886187  0.01602014 0.83876643]


In [71]:
result=pd.DataFrame({
    'respondent_id': df3['respondent_id'],
    'h1n1_vaccine':y_prob[:,1],
    'seasonal_vaccine':y_prob1[:,1]
})

In [73]:
result.to_csv('result.csv',index=False)